# SCOPE-DTI Running Demo | [Paper]() | [Repo](https://github.com/Yigang-Chen/SCOPE-DTI)

## Import required modules.

In [1]:
from configs import get_cfg_defaults
import os
import sys
from time import time
from utils import set_seed, graph_collate_func, mkdir, print_with_time
from models import Scope
from trainer import Trainer
import torch
import warnings
import pandas as pd
import numpy as np
from dataloader import DTIDataset, ConditionalInitiableDataset
from torch.utils.data import DataLoader
import pickle as pkl

/data/home/chenyigang/micromamba/envs/scope/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

The customized configuration used in this demo is stored in `configs/default.yaml`, this file overwrites defaults in `config.py` where a value is specified.

In [2]:
s = time()   # Start time

cfg = get_cfg_defaults()

cfg_path = "config_yaml/default.yaml"

assert os.path.exists(cfg_path), f"Config file {cfg_path} does not exist"

cfg.merge_from_file(cfg_path)

cfg.freeze()
mkdir(cfg.RESULT.OUTPUT_DIR)
# experiment = None
print_with_time(f"Config yaml: {cfg_path}")
print_with_time(f"Hyperparameters:")
print_with_time(dict(cfg))
set_seed(cfg.SOLVER.SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print_with_time(f"Running on: {device}")
torch.cuda.empty_cache()
warnings.filterwarnings("ignore")

[2025-03-06 03:08:22] Config yaml: config_yaml/default.yaml
[2025-03-06 03:08:22] Hyperparameters:
[2025-03-06 03:08:22] {'DRUG': CfgNode({'ATOM_IN_DIM': [74, 1], 'ATOM_HIDDEN_DIM': [320, 64], 'EDGE_IN_DIM': [16, 1], 'EDGE_HIDDEN_DIM': [32, 1], 'NUM_LAYERS': 3, 'DROP_RATE': 0.1, 'MAX_NODES': 300, 'EDGE_CUTOFF': 4.5, 'NUM_RDF': 16}), 'PROTEIN': CfgNode({'EMBEDDING_DIM': 320, 'PADDING': True, 'MAX_LENGTH': 2000, 'GRAPH': CfgNode({'COORD_PATH': 'data/demo_unip_cords.pkl', 'PATH': 'data/protein_graph.pkl', 'EDGE_CUTOFF': 10, 'NUM_KNN': 5, 'NUM_LAYER': 4, 'FC_BIAS': True})}), 'SOLVER': CfgNode({'MAX_EPOCH': 70, 'BATCH_SIZE': 16, 'NUM_WORKERS': 16, 'LR': 5e-05, 'SEED': 3076}), 'RESULT': CfgNode({'OUTPUT_DIR': './result_noname', 'SAVE_MODEL': True}), 'DECODER': CfgNode({'NAME': 'MLP', 'IN_DIM': 256, 'HIDDEN_DIM': 128, 'OUT_DIM': 64, 'BINARY': 1}), 'BCN': CfgNode({'HEADS': 2}), 'DATA': CfgNode({'TRAIN': 'data/demo_data/train.parquet', 'VAL': 'data/demo_data/val.parquet', 'TEST': 'data/demo_dat

## Dataset

We randomly selected 10 proteins along with their interaction data from the SCOPE Total dataset, utilizing a semi-inductive split strategy.

In [3]:
df_train = pd.read_parquet(cfg.DATA.TRAIN)
df_val = pd.read_parquet(cfg.DATA.VAL)
df_test = pd.read_parquet(cfg.DATA.TEST)

print_with_time("Loaded data")

dataset = ConditionalInitiableDataset(**cfg)

train_dataset = DTIDataset(df_train.index.values, df_train, father=dataset)
val_dataset = DTIDataset(df_val.index.values, df_val, father=dataset)
test_dataset = DTIDataset(df_test.index.values, df_test, father=dataset)


[2025-03-06 03:08:22] Loaded data
[2025-03-06 03:08:22] Recieved data, length: 2602
[2025-03-06 03:08:22] Loaded data, length: 2602
[2025-03-06 03:08:22] Recieved data, length: 363
[2025-03-06 03:08:22] Loaded data, length: 363
[2025-03-06 03:08:22] Recieved data, length: 725
[2025-03-06 03:08:22] Loaded data, length: 725


# Protein Graph Preprocessing

We first perform protein graph preprocessing. The protein coordinate data from the SCOPE dataset is stored in the `data` directory. To streamline the demo process, ​only 10 proteins​ are encoded using the file `data/demo_unip_cords.pkl`. For large-scale predictions, modify the configuration variable `_C.PROTEIN.GRAPH.COORD_PATH` in your config file. Running this encoding step generates protein graph files, which will be reused automatically if detected in subsequent runs to avoid redundant processing.


In [4]:
dataset._preprocessing()

[2025-03-06 03:08:22] Preprocessing protein data
[2025-03-06 03:08:22] Protein graph data already exists


# Dataloader and Model setup

Here, we load the data using the `DataLoader()` function. Then, we use the previously defined configuration to set up the model and optimizer we will subsequently train.

In [5]:
params = {'batch_size': cfg.SOLVER.BATCH_SIZE, 'shuffle': True, 
          'num_workers': cfg.SOLVER.NUM_WORKERS, 'drop_last': True, 
          'collate_fn': graph_collate_func, 'pin_memory': True}
training_generator = DataLoader(train_dataset, **params)
params['shuffle'] = False
params['drop_last'] = False
val_generator = DataLoader(val_dataset, **params)
test_generator = DataLoader(test_dataset, **params)

model = Scope(**cfg).to(device)
with open(os.path.join(cfg.RESULT.OUTPUT_DIR, "model_architecture.txt"), "w") as wf:
    wf.write(str(model))
print_with_time(f"Directory for saving result: {cfg.RESULT.OUTPUT_DIR}")

[2025-03-06 03:08:23] Directory for saving result: ./result_noname


## Model Training and Test Optimized Model

Optimize model parameters using the trainer and check test performance.

In [6]:
opt = torch.optim.Adam(model.parameters(), lr=cfg.SOLVER.LR)
if torch.cuda.is_available():
  torch.backends.cudnn.benchmark = True

trainer = Trainer(model, opt, device, training_generator, val_generator, test_generator, **cfg)
result = trainer.train()

e = time()  # End time

print(f"Total running time: {round(e - s, 2)}s")  # Print the total running time

100%|██████████| 162/162 [00:21<00:00,  7.41it/s]

[2025-03-06 03:08:45] Training at Epoch 1 with training loss 0.6993224723839465


[2025-03-06 03:08:48] Validation at Epoch 1 with validation loss 0.8176725299462028  AUROC 0.5808206686930091 AUPRC 0.5400206853596703


100%|██████████| 162/162 [00:18<00:00,  8.82it/s]

[2025-03-06 03:09:06] Training at Epoch 2 with training loss 0.6801543077568949


...

[2025-03-06 03:32:42] Training at Epoch 69 with training loss 0.41744044165552396


[2025-03-06 03:32:44] Validation at Epoch 69 with validation loss 1.253385215997696  AUROC 0.8740729483282675 AUPRC 0.8375962861907824


100%|██████████| 162/162 [00:18<00:00,  8.81it/s]

[2025-03-06 03:33:03] Training at Epoch 70 with training loss 0.4186840461176119


[2025-03-06 03:33:06] Validation at Epoch 70 with validation loss 1.2530452129633531  AUROC 0.8731003039513677 AUPRC 0.8767361867509333
[2025-03-06 03:33:10] Test at Best Model of Epoch 62 with test loss 1.1821179373756698  AUROC 0.836491772835898 AUPRC 0.833638208952054 Sensitivity 0.6629526462395543 Specificity 0.8770491803278688 Accuracy 0.7710344827586207 Thred_optim 0.4546685814857483
Total running time: 1487.85s


Experimental results​ indicate that the model requires approximately ​1500 seconds​ (25 minutes) to run on an NVIDIA GeForce RTX 2080 Ti GPU, achieving an ​AUROC of 0.84​ on the test dataset.